# ECG Split 1D-CNN Client Side

This code is the server part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages

In [1]:
import os
import struct
import socket
import pickle
import time

import h5py
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD

## Define ECG dataset class

In [9]:
class ECG(Dataset):
    def __init__(self, train=True):
        if train:
            with h5py.File(os.path.join('train_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(os.path.join('test_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

### Set batch size

In [10]:
batch_size = 32

## Make train and test dataset batch generator

In [11]:
train_dataset = ECG(train=True)
test_dataset = ECG(train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

### Total number of batches

In [12]:
total_batch = len(train_loader)
print(total_batch)

414


## Define ECG client model
Client side has only **2 convolutional layers**.

In [13]:
class EcgClient(nn.Module):
    def __init__(self):
        super(EcgClient, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu3 = nn.LeakyReLU()
        self.pool3 = nn.MaxPool1d(2)  # 32 x 16
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = x.view(-1, 32 * 16)
        return x

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

### Set random seed

In [15]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

### Assign intial weight as same as non-split model

In [16]:
ecg_client = EcgClient()
ecg_client.to(device)

checkpoint = torch.load("init_weight.pth")
ecg_client.conv1.weight.data = checkpoint["conv1.weight"]
ecg_client.conv1.bias.data = checkpoint["conv1.bias"]
ecg_client.conv2.weight.data = checkpoint["conv2.weight"]
ecg_client.conv2.bias.data = checkpoint["conv2.bias"]
ecg_client.conv3.weight.data = checkpoint["conv3.weight"]
ecg_client.conv3.bias.data = checkpoint["conv3.bias"]

### Set other hyperparameters in the model
Hyperparameters here should be same with the server side.

In [18]:
epoch = 180
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = Adam(ecg_client.parameters(), lr=lr)

## Socket initialization

### Required socket functions

In [19]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

### Set host address and port number

In [24]:
host = 'localhost'
port = 9000
max_recv = 4096

### Open the client socket

In [25]:
s = socket.socket()
s.connect((host, port))

## Real training process

In [26]:
for e in range(epoch):
    print("Epoch {} - ".format(e+1), end='')
    
    for _, batch in enumerate(train_loader):
        x, label = batch
        x, label = x.to(device), label.to(device)
        optimizer.zero_grad()
        output = ecg_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        msg = pickle.dumps(msg)
        send_msg(s, msg)
        msg = recv_msg(s)
        client_grad = pickle.loads(msg)
        output.backward(client_grad)
        optimizer.step()
            
    with torch.no_grad():  # calculate test accuracy
        for _, batch in enumerate(test_loader):
            x, label = batch
            x, label = x.to(device), label.to(device)
            client_output = ecg_client(x)
            msg = {
                'client_output': client_output,
                'label': label
            }
            msg = pickle.dumps(msg)
            send_msg(s, msg)
    
    msg = recv_msg(s)
    train_test_status = pickle.loads(msg)
    print(train_test_status)

Epoch 1 - loss: 1.3045, acc: 60.93% / test_loss: 1.1231, test_acc: 78.94%
Epoch 2 - loss: 1.0827, acc: 82.80% / test_loss: 1.0436, test_acc: 86.43%
Epoch 3 - loss: 1.0493, acc: 85.72% / test_loss: 1.0279, test_acc: 87.74%
Epoch 4 - loss: 1.0423, acc: 86.31% / test_loss: 1.0313, test_acc: 87.40%
Epoch 5 - loss: 1.0383, acc: 86.70% / test_loss: 1.0206, test_acc: 88.36%
Epoch 6 - loss: 1.0334, acc: 87.15% / test_loss: 1.0238, test_acc: 88.07%
Epoch 7 - loss: 1.0318, acc: 87.30% / test_loss: 1.0347, test_acc: 86.88%
Epoch 8 - loss: 1.0276, acc: 87.66% / test_loss: 1.0184, test_acc: 88.57%
Epoch 9 - loss: 1.0273, acc: 87.78% / test_loss: 1.0297, test_acc: 87.58%
Epoch 10 - loss: 1.0308, acc: 87.34% / test_loss: 1.0223, test_acc: 88.22%
Epoch 11 - loss: 1.0259, acc: 87.84% / test_loss: 1.0386, test_acc: 86.59%
Epoch 12 - loss: 1.0241, acc: 88.01% / test_loss: 1.0182, test_acc: 88.60%
Epoch 13 - loss: 1.0225, acc: 88.16% / test_loss: 1.0327, test_acc: 87.23%
Epoch 14 - loss: 1.0216, acc: 88.2

Epoch 111 - loss: 0.9986, acc: 90.28% / test_loss: 0.9988, test_acc: 90.31%
Epoch 112 - loss: 0.9969, acc: 90.43% / test_loss: 1.0094, test_acc: 89.20%
Epoch 113 - loss: 1.0018, acc: 89.97% / test_loss: 1.0000, test_acc: 90.25%
Epoch 114 - loss: 0.9973, acc: 90.42% / test_loss: 0.9946, test_acc: 90.69%
Epoch 115 - loss: 0.9998, acc: 90.15% / test_loss: 0.9974, test_acc: 90.40%
Epoch 116 - loss: 0.9980, acc: 90.33% / test_loss: 0.9950, test_acc: 90.65%
Epoch 117 - loss: 0.9965, acc: 90.48% / test_loss: 0.9945, test_acc: 90.66%
Epoch 118 - loss: 0.9955, acc: 90.57% / test_loss: 0.9942, test_acc: 90.70%
Epoch 119 - loss: 0.9957, acc: 90.56% / test_loss: 0.9943, test_acc: 90.69%
Epoch 120 - loss: 0.9966, acc: 90.46% / test_loss: 0.9943, test_acc: 90.72%
Epoch 121 - loss: 0.9965, acc: 90.47% / test_loss: 0.9972, test_acc: 90.48%
Epoch 122 - loss: 0.9965, acc: 90.50% / test_loss: 0.9957, test_acc: 90.62%
Epoch 123 - loss: 0.9970, acc: 90.45% / test_loss: 0.9973, test_acc: 90.41%
Epoch 124 - 

In [35]:
print('Finished Training!')
print('Result is on the server side.')

Finished Training!
Result is on the server side.
